In [2]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import json

In [5]:
constFile=pd.read_csv('Constellation Names.csv')

In [6]:
constFile.head()

,Name,Abbreviation,Genitive,Meaning,Brightest Star
0,Andromeda,And,Andromedae,Princess of Ethiopia,Alpheratz
1,Antlia,Ant,Antilae,Air Pump,Alpha Antilae
2,Apus,Aps,Apodis,Bird of Paradise,Alpha Apodis
3,Aquarius,Aqr,Aquarii,Water-bearer,Sadalsuud
4,Aquila,Aql,Aquilae,The Eagle,Altair


In [7]:
constFile['Name']

0      Andromeda
1         Antlia
2           Apus
3       Aquarius
4         Aquila
         ...    
83    Ursa Minor
84          Vela
85         Virgo
86        Volans
87     Vulpecula
Name: Name, Length: 88, dtype: object

In [8]:
# Converting Name column according to wikipedia url
def convertName(name):
    includesSpace= False
    spaceIndex=0
    for i in range(0,len(name)):
        if(name[i]==' '):
            includesSpace=True
            spaceIndex=i
    if (includesSpace):
        return name[:spaceIndex]+'_'+name[spaceIndex+1:]
    else:
        return name + '_(constellation)'
        
    
    

In [9]:
# Three new columns for scrapping information and image urls
constFile['UrlName']=constFile['Name']
constFile['aName']=constFile['Name']
constFile['imgUrl']=constFile['aName']

In [10]:
# Exceptional 'Bootes'
constFile['aName'][8]='Bootes'

In [11]:
for i in range(0,len(constFile['Name'])):
    constFile['UrlName'][i]=convertName(constFile['Name'][i])

In [12]:
#for i in constFile['UrlName']:
#    print(i)

Andromeda_(constellation)
Antlia_(constellation)
Apus_(constellation)
Aquarius_(constellation)
Aquila_(constellation)
Ara_(constellation)
Aries_(constellation)
Auriga_(constellation)
Boötes_(constellation)
Caelum_(constellation)
Camelopardalis_(constellation)
Cancer_(constellation)
Canes_Venatici
Canis_Major
Canis_Minor
Capricornus_(constellation)
Carina_(constellation)
Cassiopeia_(constellation)
Centaurus_(constellation)
Cepheus_(constellation)
Cetus_(constellation)
Chamaeleon_(constellation)
Circinus_(constellation)
Columba_(constellation)
Coma_Berenices
Corona_Australis
Corona_Borealis
Corvus_(constellation)
Crater_(constellation)
Crux_(constellation)
Cygnus_(constellation)
Delphinus_(constellation)
Dorado_(constellation)
Draco_(constellation)
Equuleus_(constellation)
Eridanus_(constellation)
Fornax_(constellation)
Gemini_(constellation)
Grus_(constellation)
Hercules_(constellation)
Horologium_(constellation)
Hydra_(constellation)
Hydrus_(constellation)
Indus_(constellation)
Lacerta

In [32]:
# For Boötes_(constellation) there is an exception as 'ascii' codec can't encode character ö

constFile['UrlName'][8]='Bootes'
constFile['UrlName'][58]='Ophiucus'

In [33]:
# for saving the wiki description url
request_body="https://en.wikipedia.org/wiki/"
#For saving the image url
head = 'https://commons.wikimedia.org/wiki/File:'

In [34]:
# This function takes constellation name and return web url for an image of that constellation from wikipedia
def imgUrl(aName):
    head = 'https://commons.wikimedia.org/wiki/File:'
    
    name=aName
    print(name)
    firstName=name.replace(' ','')
    url=head + firstName + 'CC.jpg'
    print('First try')
    try:
        page = urllib.request.urlopen(url)
        soup = BeautifulSoup(page, 'html.parser')
        sImg =soup.find('img')
        print('image found at: ',sImg['src'] )
        return sImg['src']
    except:
        print('1st try failed')
        print('2nd try..')
        name = name.replace(' ','_')
        secondUrl = head + 'Constellation_' +name +'.jpg'
        # two kind of pattern for images, if first try results in a 404 try with another url
        try:
        
            page = urllib.request.urlopen(secondUrl)
            soup = BeautifulSoup(page, 'html.parser')
            sImg =soup.find('img')
            print(sImg['src'])
            return sImg['src']
        except:
            print('2nd try failed')
            return None
        


    

In [35]:
# function for creating a dictonary of dictonaries for all the constellation's info, name, urls etc.

def jsonObjectCreate():
    jsonlist=[]
    for i in range(0,len(constFile['UrlName'])):
        searchUrl=request_body+constFile['UrlName'][i]
        request_result=urllib.request.urlopen(searchUrl)
        page=BeautifulSoup(request_result, 'html.parser')

        id =i
        name=constFile['Name'][i]
        englishName=constFile['Meaning'][i]
        brightestStar=constFile['Brightest Star'][i]
        paragraph0=page.find_all('p')[0].getText()
        paragraph1=page.find_all('p')[1].getText()
        paragraph2=page.find_all('p')[2].getText()
        

        dic={
            'id':id,
            'name':name,
            'englishName':englishName,
            'brightestStar':brightestStar,
            'paragraph0':paragraph0,
            'paragraph1':paragraph1,
            'paragraph2':paragraph2,
            'url':searchUrl,
            'imgUrl':imgUrl(constFile['aName'][i])
            }
        jsonlist.append(dic)
    return jsonlist

        
        
        


In [36]:
jsonList=jsonObjectCreate()

Andromeda
First try
image found at:  https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/AndromedaCC.jpg/402px-AndromedaCC.jpg?20230802112629
Antlia
First try
1st try failed
2nd try..
https://upload.wikimedia.org/wikipedia/commons/3/3c/Constellation_Antlia.jpg?20230802112909
Apus
First try
1st try failed
2nd try..
https://upload.wikimedia.org/wikipedia/commons/thumb/8/80/Constellation_Apus.jpg/397px-Constellation_Apus.jpg?20230802113131
Aquarius
First try
image found at:  https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/AquariusCC.jpg/431px-AquariusCC.jpg?20230802120117
Aquila
First try
image found at:  https://upload.wikimedia.org/wikipedia/commons/1/1e/AquilaCC.jpg?20230802144204
Ara
First try
1st try failed
2nd try..
https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Constellation_Ara.jpg/400px-Constellation_Ara.jpg?20230802144403
Aries
First try
image found at:  https://upload.wikimedia.org/wikipedia/commons/thumb/9/99/AriesCC.jpg/440px-AriesCC.jpg?20230802144

Norma
First try
1st try failed
2nd try..
https://upload.wikimedia.org/wikipedia/commons/5/55/Constellation_Norma.jpg?20130522131948
Octans
First try
1st try failed
2nd try..
https://upload.wikimedia.org/wikipedia/commons/thumb/7/7c/Constellation_Octans.jpg/399px-Constellation_Octans.jpg?20230802222201
Ophiucus
First try
1st try failed
2nd try..
2nd try failed
Orion
First try
image found at:  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f5/OrionCC.jpg/422px-OrionCC.jpg?20230802222523
Pavo
First try
1st try failed
2nd try..
https://upload.wikimedia.org/wikipedia/commons/thumb/9/98/Constellation_Pavo.jpg/399px-Constellation_Pavo.jpg?20230802222654
Pegasus
First try
image found at:  https://upload.wikimedia.org/wikipedia/commons/2/2c/PegasusCC.jpg?20230802222810
Perseus
First try
image found at:  https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/PerseusCC.jpg/400px-PerseusCC.jpg?20230802222940
Phoenix
First try
1st try failed
2nd try..
https://upload.wikimedia.org/wikiped

In [38]:
json_object=json.dumps(jsonList, indent=4)

with open("output.json", "w") as outfile:
    outfile.write(json_object)